# Panda's Advanced Dataframes Exercises
<hr style="border:2px solid red"> </hr>


In [1]:
import pandas as pd
import numpy as np

In [2]:
#this will be what we use every time we want to talk to a database in SQL
def get_db_url(database_name):
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{database_name}'

In [3]:
sql_query_example = """SELECT * from departments"""

In [4]:
database = get_db_url("employees") #DO NOT PRINT THIS! It will show your password!

In [5]:
df = pd.read_sql(sql_query_example , database)
df.head()

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing


### 5. Once you have successfully run a query:
#### a. Intentionally make a typo in the database url. What kind of error message do you see?



In [6]:
error_database = get_db_url("duckduckgoose") #DO NOT PRINT THIS! It will show your password!

#### b. Intentionally make an error in your SQL query. What does the error message look like?

In [7]:
#sql_query_error = """ SELECT ** FROM employees"""
#df = pd.read_sql(sql_query_error , database)
#df.head()

### 6. Read the employees and titles tables into two separate DataFrames.



In [8]:
employees_table = pd.read_sql("""SELECT * FROM employees""" , get_db_url("employees"))
titles_table = pd.read_sql("""SELECT * FROM titles""" , get_db_url("employees"))

### 7. How many rows and columns do you have in each DataFrame? Is that what you expected?

In [9]:
employees_table.shape 

(300024, 6)

In [10]:
titles_table.shape #yes, there are some employees whom have held more than one title while working for this company, so there are more title entries than employees

(443308, 4)

### 8. Display the summary statistics for each DataFrame.



In [11]:
employees_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300024 entries, 0 to 300023
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   emp_no      300024 non-null  int64 
 1   birth_date  300024 non-null  object
 2   first_name  300024 non-null  object
 3   last_name   300024 non-null  object
 4   gender      300024 non-null  object
 5   hire_date   300024 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


In [12]:
titles_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB


### 9. How many unique titles are in the `titles` DataFrame?



In [13]:
#python
titles_table.title.unique().size

7

In [14]:
#using a SQL query
unique_titles = pd.read_sql("""SELECT DISTINCT title FROM titles""" , get_db_url("employees"))
unique_titles

,title
0,Senior Engineer
1,Staff
2,Engineer
3,Senior Staff
4,Assistant Engineer
5,Technique Leader
6,Manager


### 10. What is the oldest date in the to_date column?



In [15]:
#oldest is being interpreted as the date furthest back from today
#python
titles_table.to_date.min()

datetime.date(1985, 3, 1)

In [16]:
#using a SQL query
oldest_to_date = pd.read_sql("""SELECT *
                                FROM titles
                                ORDER BY to_date
                                LIMIT 1""" ,
                                get_db_url("employees"))
oldest_to_date

,emp_no,title,from_date,to_date
0,20869,Engineer,1985-02-17,1985-03-01


### 11. What is the most recent date in the to_date column?



In [17]:
test_date = titles_table.to_date.head(1)
[test_date == titles_table.to_date.max()]

[0    True
 Name: to_date, dtype: bool]

In [18]:
titles_table.to_date.max()

datetime.date(9999, 1, 1)

In [19]:
#most recent is being interpreted as the date closest to today without going past

#python
titles_table[titles_table.to_date != titles_table.to_date.max()].to_date.max()


datetime.date(2002, 8, 1)

In [20]:
#using a SQL query
recent_to_date = pd.read_sql("""SELECT *
                                FROM titles
                                WHERE year(to_date) != 9999 
                                ORDER BY to_date DESC""" ,
                                get_db_url("employees"))
recent_to_date.head()

,emp_no,title,from_date,to_date
0,495525,Staff,1996-08-01,2002-08-01
1,446732,Senior Engineer,1992-11-10,2002-08-01
2,447596,Staff,1996-08-01,2002-08-01
3,448129,Senior Staff,2001-01-23,2002-08-01
4,452819,Engineer,1993-08-01,2002-08-01


In [21]:
#lesson notes

In [22]:
titles_table.emp_no.agg(["median"])

median    249847.5
Name: emp_no, dtype: float64

In [23]:
titles_table.groupby("title").agg(["median"])

,emp_no
,median
title,
Assistant Engineer,249437.5
Engineer,249735.0
Manager,110646.0
Senior Engineer,249845.0
Senior Staff,249853.0
Staff,250242.0
Technique Leader,248527.0


In [24]:
titles_table.groupby("title").agg(["mean","median"]).T

title          Assistant Engineer       Engineer        Manager  \
emp_no mean         251495.398533  252943.159987  110780.833333   
       median       249437.500000  249735.000000  110646.000000   

title          Senior Engineer   Senior Staff          Staff  Technique Leader  
emp_no mean      253034.375949  253423.367183  253399.240672     251708.951052  
       median    249845.000000  249853.000000  250242.000000     248527.000000

In [25]:
titles_table.groupby("title").size()

title
Assistant Engineer     15128
Engineer              115003
Manager                   24
Senior Engineer        97750
Senior Staff           92853
Staff                 107391
Technique Leader       15159
dtype: int64

In [26]:
students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [27]:
# Construct the DataFrame using the above lists and arrays.

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

In [28]:
df

,name,math,english,reading,classroom
0,Sally,93,64,75,A
1,Jane,81,60,73,B
2,Suzie,84,71,82,B
3,Billy,84,92,60,A
4,Ada,90,70,73,B
5,John,88,87,96,A
6,Thomas,78,66,96,A
7,Marie,62,69,67,B
8,Albert,61,60,75,A
9,Richard,94,61,83,A


## Exercises II
### 1. Copy the `users` and `roles` DataFrames from the curriculum.



In [29]:
# the users DataFrame.
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [30]:
# the roles DataFrame
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


### 2. What is the result of using a right join on the DataFrames?



In [31]:
#answer
users.merge(roles, left_on='role_id', right_on='id', how='right')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


In [32]:
#showing the merge row for deeper understanding
users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


### 3. What is the result of using an outer join on the DataFrames?



In [33]:
#answer
users.merge(roles, left_on='role_id', right_on='id', how='outer')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


In [34]:
#showing the merge row for deeper understanding
users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


### 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?



In [35]:
#no keys looks like theres no data!
users.merge(roles)

,id,name,role_id


### 5. Load the mpg dataset from PyDataset.



In [164]:
import pydataset
from pydataset import data

In [165]:
mpg = data('mpg')

### 6. Output and read the documentation for the mpg dataset.



In [166]:
mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   manufacturer  234 non-null    object 
 1   model         234 non-null    object 
 2   displ         234 non-null    float64
 3   year          234 non-null    int64  
 4   cyl           234 non-null    int64  
 5   trans         234 non-null    object 
 6   drv           234 non-null    object 
 7   cty           234 non-null    int64  
 8   hwy           234 non-null    int64  
 9   fl            234 non-null    object 
 10  class         234 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 21.9+ KB


In [167]:
#instructor solution
pydataset.data("mpg", show_doc = True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




### 7. How many rows and columns are in the dataset?



In [168]:
mpg.shape

(234, 11)

### 8. Check out your column names and perform any cleanup you may want on them.



In [169]:
mpg = mpg.rename(columns={'cty': 'city'})
mpg = mpg.rename(columns={'hwy': 'highway'})
mpg = mpg.rename(columns={'trans': 'transmission'})
mpg

,manufacturer,model,displ,year,cyl,transmission,drv,city,highway,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


### 9. Display the summary statistics for the dataset.



In [170]:
mpg.describe()

,displ,year,cyl,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


### 10. How many different manufacturers are there?



In [171]:
mpg.manufacturer.unique().size

15

### 11. How many different models are there?



In [172]:
mpg.model.unique().size

38

### 12. Create a column named `mileage_difference` like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.



In [173]:
mpg = mpg.assign(mileage_difference = mpg.highway - mpg.city)
mpg

,manufacturer,model,displ,year,cyl,transmission,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


In [175]:
#instructor solution
mpg['milage_diff']= mpg.highway - mpg.city
mpg

,manufacturer,model,displ,year,cyl,transmission,drv,city,highway,fl,class,mileage_difference,milage_diff
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,8


### 13. Create a column named `average_mileage` like you did in the DataFrames exercises; this is the mean of the city and highway mileage.



In [176]:
mpg = mpg.assign(average_mileage = (mpg.highway + mpg.city)/2 )
mpg

,manufacturer,model,displ,year,cyl,transmission,drv,city,highway,fl,class,mileage_difference,milage_diff,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,8,22.0


### 14. Create a new column on the mpg dataset named `is_automatic` that holds boolean values denoting whether the car has an automatic transmission.



In [177]:
#theres more than one type of automatic transmission, so I'm gonna isolate by using .str.contains()
mpg = mpg.assign(is_automatic = mpg.transmission.str.contains("auto"))
mpg

,manufacturer,model,displ,year,cyl,transmission,drv,city,highway,fl,class,mileage_difference,milage_diff,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,8,22.0,False


In [179]:
#instructor solution
mpg["is_automatic"] = mpg.transmission.apply(lambda value : value[0] == 'a')

### 15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [180]:
#checking what each manufacturer looks like
mpg.groupby("manufacturer")["average_mileage"].agg(["median"])

,median
manufacturer,
audi,21.50
chevrolet,19.00
dodge,15.00
ford,15.50
honda,28.50
hyundai,22.50
jeep,16.00
land rover,14.00
lincoln,14.00


In [196]:
#isolating the manufacturer  with best average mpg
mpg.groupby("manufacturer").average_mileage.agg("median").nlargest(1)


manufacturer
honda    28.5
Name: average_mileage, dtype: float64

### 16. Do automatic or manual cars have better miles per gallon?



In [49]:
#if we group by transmission then we will get a list of all the different automatic transmissions and different manual transmissions
#if we group by is_automatic tho we get auto vs manual no matter their type

mpg.groupby("is_automatic")["average_mileage"].agg(["median"])

#it looks like Manual Trnasmission actually has better miles per gallon

,median
is_automatic,
False,22.0
True,19.0


In [50]:
#lesson notes

In [51]:
df

,name,math,english,reading,classroom
0,Sally,93,64,75,A
1,Jane,81,60,73,B
2,Suzie,84,71,82,B
3,Billy,84,92,60,A
4,Ada,90,70,73,B
5,John,88,87,96,A
6,Thomas,78,66,96,A
7,Marie,62,69,67,B
8,Albert,61,60,75,A
9,Richard,94,61,83,A


In [ ]:
pd.crosstab(# rows df.passing_math, #columns df.classroom)

In [ ]:
pd.crosstab(# rows df.passing_math, #columns df.classroom, normalize=True).round(3)

In [ ]:
df.pivot_table(index='classroom', columns='passing_math', values='math')

In [ ]:
df.pivot_table(aggfunc=[]"min","median","max"], index="classroom", columns="passing_math")

In [ ]:
mpg.pivot_table(values= "highway", index="Manufacturer" , columns="class", value="" )

## Exercises III
### 1. Use your `get_db_url` function to help you explore the data from the chipotle database.

In [53]:
chipotle = pd.read_sql( "SELECT * FROM orders" , get_db_url("chipotle"))
chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


In [66]:
chipotle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   4622 non-null   int64  
 1   order_id             4622 non-null   int64  
 2   quantity             4622 non-null   int64  
 3   item_name            4622 non-null   object 
 4   choice_description   4622 non-null   object 
 5   item_price           4622 non-null   object 
 6   chipotle_item_price  4622 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 252.9+ KB


In [67]:
chipotle.shape

(4622, 7)

In [69]:
chipotle_menu = chipotle["item_name"].unique()
chipotle_menu

array(['Chips and Fresh Tomato Salsa', 'Izze', 'Nantucket Nectar',
       'Chips and Tomatillo-Green Chili Salsa', 'Chicken Bowl',
       'Side of Chips', 'Steak Burrito', 'Steak Soft Tacos',
       'Chips and Guacamole', 'Chicken Crispy Tacos',
       'Chicken Soft Tacos', 'Chicken Burrito', 'Canned Soda',
       'Barbacoa Burrito', 'Carnitas Burrito', 'Carnitas Bowl',
       'Bottled Water', 'Chips and Tomatillo Green Chili Salsa',
       'Barbacoa Bowl', 'Chips', 'Chicken Salad Bowl', 'Steak Bowl',
       'Barbacoa Soft Tacos', 'Veggie Burrito', 'Veggie Bowl',
       'Steak Crispy Tacos', 'Chips and Tomatillo Red Chili Salsa',
       'Barbacoa Crispy Tacos', 'Veggie Salad Bowl',
       'Chips and Roasted Chili-Corn Salsa',
       'Chips and Roasted Chili Corn Salsa', 'Carnitas Soft Tacos',
       'Chicken Salad', 'Canned Soft Drink', 'Steak Salad Bowl',
       '6 Pack Soft Drink', 'Chips and Tomatillo-Red Chili Salsa', 'Bowl',
       'Burrito', 'Crispy Tacos', 'Carnitas Crispy Tacos

In [70]:
chipotle_menu = pd.Series(chipotle_menu)
chipotle_menu.sort_values()
#dirty data! The menu has doubles! Will keep an eye on this for future Q's

35                        6 Pack Soft Drink
18                            Barbacoa Bowl
13                         Barbacoa Burrito
27                    Barbacoa Crispy Tacos
45                      Barbacoa Salad Bowl
22                      Barbacoa Soft Tacos
16                            Bottled Water
37                                     Bowl
38                                  Burrito
12                              Canned Soda
33                        Canned Soft Drink
15                            Carnitas Bowl
14                         Carnitas Burrito
40                    Carnitas Crispy Tacos
49                           Carnitas Salad
44                      Carnitas Salad Bowl
31                      Carnitas Soft Tacos
4                              Chicken Bowl
11                          Chicken Burrito
9                      Chicken Crispy Tacos
32                            Chicken Salad
20                       Chicken Salad Bowl
10                       Chicken

### 2. What is the total price for each order?



In [61]:
#looks like this code is struggling because item_price isn't a number, but rather an object/string
chipotle.groupby("order_id")["item_price"].agg(["sum"])

,sum
order_id,
1,$2.39 $3.39 $3.39 $2.39
2,$16.98
3,$10.98 $1.69
4,$11.75 $9.25
5,$9.25 $4.45
...,...
1830,$11.75 $11.25
1831,$9.25 $2.15 $1.50
1832,$8.75 $4.45


In [62]:
#converting prices to a float python can read
chipotle_item_prices = chipotle.item_price.str.replace("$","").str.replace(",","").astype("float64")
chipotle_item_prices

<ipython-input-62-2d40e777b02e>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  chipotle_item_prices = chipotle.item_price.str.replace("$","").str.replace(",","").astype("float64")


0        2.39
1        3.39
2        3.39
3        2.39
4       16.98
        ...  
4617    11.75
4618    11.75
4619    11.25
4620     8.75
4621     8.75
Name: item_price, Length: 4622, dtype: float64

In [63]:
#using the code above to create a new column (i do not wish to delete or replace the item_price column at this time)
chipotle = chipotle.assign( chipotle_item_price = chipotle.item_price.str.replace("$","").str.replace(",","").astype("float64") )

chipotle

<ipython-input-63-57fb5d404975>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  chipotle = chipotle.assign( chipotle_item_price = chipotle.item_price.str.replace("$","").str.replace(",","").astype("float64") )


,id,order_id,quantity,item_name,choice_description,item_price,chipotle_item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39,2.39
1,2,1,1,Izze,[Clementine],$3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98,16.98
...,...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75,11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75,11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25,11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75,8.75


In [64]:
#Okay! lets try again, but this time with our cleaned up column of chipotle_item_price's
chipotle.groupby("order_id")["chipotle_item_price"].agg(["sum"])

,sum
order_id,
1,11.56
2,16.98
3,12.67
4,21.00
5,13.70
...,...
1830,23.00
1831,12.90
1832,13.20


### 3. What are the most popular 3 items?



In [118]:
#most popular defined as the item with the highest quantity sold
chipotle.groupby("item_name")["quantity"].agg(["sum"]).nlargest(3,"sum")

,sum
item_name,
Chicken Bowl,761
Chicken Burrito,591
Chips and Guacamole,506


### 4. Which item has produced the most revenue?



In [127]:
#we can't multiply the item_price by quantity, because the item_prices are NOT consistent
#therefore grouping by the item name and doing a total sum of all it's prices is the way to go!
chipotle.groupby("item_name")["chipotle_item_price"].agg(["sum"]).nlargest(1,"sum")

,sum
item_name,
Chicken Bowl,7342.73


### 5. Join the `employees` and `titles` DataFrames together.



In [128]:
employees_table.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [129]:
titles_table.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


In [131]:
employees_with_titles = employees_table.merge(titles_table, left_on='emp_no', right_on='emp_no', how='inner')
employees_with_titles

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


### 6. For each title, find the hire date of the employee that was hired most recently with that title.

In [133]:
employees_with_titles.hire_date.max()

datetime.date(2000, 1, 28)

In [140]:
employees_with_titles.groupby("title")["hire_date"].max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

In [197]:
employees_with_titles.groupby("title")["hire_date"].agg("max")

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

### 7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)



In [141]:
departments = pd.read_sql("""SELECT * FROM departments""" , get_db_url("employees"))
dept_emp = pd.read_sql("""SELECT * FROM dept_emp""" , get_db_url("employees"))

In [142]:
departments.head()

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing


In [143]:
dept_emp.head()

,emp_no,dept_no,from_date,to_date
0,10001,d005,1986-06-26,9999-01-01
1,10002,d007,1996-08-03,9999-01-01
2,10003,d004,1995-12-03,9999-01-01
3,10004,d004,1986-12-01,9999-01-01
4,10005,d003,1989-09-12,9999-01-01


In [144]:
employees_with_titles_deptno = employees_with_titles.merge(dept_emp, left_on='emp_no', right_on='emp_no', how='inner')
employees_with_titles_deptno

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date_x,to_date_x,dept_no,from_date_y,to_date_y
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01,d007,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01,d004,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01,d004,1986-12-01,9999-01-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01,d004,1986-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
489898,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29,d005,1987-08-30,9999-01-01
489899,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01,d005,1987-08-30,9999-01-01
489900,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01,d002,1993-12-27,9999-01-01
489901,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27,d002,1993-12-27,9999-01-01


In [146]:
employees_with_titles_deptno_department = employees_with_titles_deptno.merge(departments, left_on='dept_no', right_on='dept_no', how='inner')
employees_with_titles_deptno_department

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date_x,to_date_x,dept_no,from_date_y,to_date_y,dept_name
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,1986-06-26,9999-01-01,Development
1,10006,1953-04-20,Anneke,Preusig,F,1989-06-02,Senior Engineer,1990-08-05,9999-01-01,d005,1990-08-05,9999-01-01,Development
2,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15,Assistant Engineer,1998-03-11,2000-07-31,d005,1998-03-11,2000-07-31,Development
3,10012,1960-10-04,Patricio,Bridgland,M,1992-12-18,Engineer,1992-12-18,2000-12-18,d005,1992-12-18,9999-01-01,Development
4,10012,1960-10-04,Patricio,Bridgland,M,1992-12-18,Senior Engineer,2000-12-18,9999-01-01,d005,1992-12-18,9999-01-01,Development
...,...,...,...,...,...,...,...,...,...,...,...,...,...
489898,499975,1952-11-09,Masali,Chorvat,M,1992-01-23,Staff,1995-11-21,1997-07-23,d002,1996-10-19,1997-07-23,Finance
489899,499977,1956-06-05,Martial,Weisert,F,1996-09-17,Staff,1999-12-28,9999-01-01,d002,1999-12-28,9999-01-01,Finance
489900,499989,1954-05-26,Keiichiro,Lindqvist,M,1993-10-28,Senior Staff,1996-10-29,2001-03-07,d002,1996-10-29,2001-03-07,Finance
489901,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01,d002,1993-12-27,9999-01-01,Finance


In [148]:
#this includes employees no longer working here! oops! use a sql query
pd.crosstab(employees_with_titles_deptno_department.dept_name , employees_with_titles_deptno_department.title).T

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,298,7769,0,0,0,6445,1831,378,0
Engineer,2362,58135,0,0,0,49649,13852,2986,0
Manager,4,2,2,2,2,4,4,2,2
Senior Engineer,2027,49326,0,0,0,42205,11864,2570,0
Senior Staff,13925,1247,12139,12274,13940,1270,0,11637,36191
Staff,16150,1424,13929,14342,16196,1478,0,13495,41808
Technique Leader,309,7683,0,0,0,6557,1795,393,0
